In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb
%run vis/helpersFunctions/barCharts.ipynb

***vis_completables_times* function**:
* Input: players_info dictionary
* Displays bar chart showing for each completable the maximum, minimum and mean time of completion by players

In [ ]:
def vis_completables_times():
    # find full list of completables
    out=interactive(construct_completable_time_bar_chart,
                        xaxis=xaxis)
    md_text=widgets.HTML("Bar chart with max and min completion time in each completable")
    return widgets.VBox([md_text,out])

    
def construct_completable_time_bar_chart(xaxis, ):
    #get alls players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    completablesSelectedCompletableTime=getSublistSelectedFromUnionListSelected(completables_times_list, checkboxesCompletableSelected.get_selected_options())
    # max, min time per completable
    max_times = []
    min_times = []
    for c in completablesSelectedCompletableTime:
        maximum = 0
        minimum = float('Inf')
        for player in playersSelected:
            if c in players_info[player]["completables_times"].keys():
                times = players_info[player]["completables_times"][c]
                try:
                    if len(times)==2:
                        end_time = times[1]
                        start_time = times[0]
                        raw_total_time = end_time-start_time
                        total_time = raw_total_time.days*24*60+raw_total_time.seconds/60.0 # minutes
                        # update values
                        if total_time > maximum:
                            maximum = total_time
                        if total_time < minimum:
                            minimum = total_time
                except:
                    pass
        if minimum==float('Inf'):
            minimum=maximum
        max_times.append(maximum)
        min_times.append(minimum)
    N=len(completablesSelectedCompletableTime)
    n_groups=N
    if xaxis==0:  #getAllDataInAListOfList
        completablesSelectedCompletableTime=[completablesSelectedCompletableTime]
        min_times=[min_times]
        max_times=[max_times]
        nbgraphshown=1
    elif xaxis==-1:  #getnbFirstData
        nb=10
        nbgraphshown=1
        completablesSelectedCompletableTime=[completablesSelectedCompletableTime[:nb]]
        max_times=[max_times[:nb]]
        min_times=[min_times[:nb]]
        if N>nb:
            N=nb
    elif xaxis==-2:  #getnbLastData
        nb=10
        nbgraphshown=1
        completablesSelectedCompletableTime=[completablesSelectedCompletableTime[-nb:]]
        max_times=[max_times[-nb:]]
        min_times=[min_times[-nb:]]
        if N>nb:
            N=nb
    else:   #getxaxisDataSeparatedInListOfLists
        completablesSelectedCompletableTime=separateListToSubListEachXvalues(completablesSelectedCompletableTime,xaxis)
        max_times=separateListToSubListEachXvalues(max_times, xaxis)
        min_times=separateListToSubListEachXvalues(min_times, xaxis)
        N=xaxis
        nbgraphshown=len(max_times)
    # Bar chart showing for each completable the maximum, minimum and mean time of completion by players.
    for n in range(nbgraphshown):
        if n_groups>0 :
            if n<nbgraphshown-1:
                index = np.arange(N)
            else:
                index = np.arange(len(completablesSelectedCompletableTime[n]))
            bar_width = 0.35
            times=[max_times,min_times]
            labels=['max', 'min']
            max=0
            for t in times[0][n]:
                if t>max:
                    max=t
            max+=2
            for i in range(len(labels)):
                bar=plt.bar(index+i*bar_width, times[i][n], bar_width, 
                        label=labels[i],
                        tick_label=completablesSelectedCompletableTime[n])
                ticks_label=completablesSelectedCompletableTime[n]
                add_value_labels(bar)
            plt.ylim(0,max)
            plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            plt.gca().axes.set_xticks([x+bar_width/2 for x in range(len(ticks_label))])
            plt.gca().axes.set_xticklabels(ticks_label)
        else:
            noDataToFillVis(100)
        plt.xlabel("Completable")
        plt.ylabel("Completion time (in minutes)")
        plt.xticks(rotation=90)
        plt.title("Max and min completion times per completable")
        plt.show()